<a href="https://colab.research.google.com/github/KeniaEsparza/TI_1_Practica3_EsparzaKenia/blob/main/TI_1_Practica3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica 3

**Nombre:** Esparza Duran Kenia Jaqueline

**e-mail:** kenia.esparza5866@alumnos.udg.mx

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## MODULES

In [2]:
import math
import numpy as np
import pandas as pd

import plotly.graph_objects as go

from scipy.stats import wrapcauchy
from scipy.stats import levy_stable

from scipy.spatial import distance

## CLASSES

In [3]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y

    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)

    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)

    # rotate vector
    def rotated(self, angle):
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

## FUNCTIONS

In [4]:
###############################################################################################
# Turning angle
# This function calculates the turning angle between three consecutive positions
###############################################################################################
def turning_angle(pos_a, pos_b, pos_c):
    """
    Arguments:
        pos_a: First position coordinates
        pos_b: Second position coordinates
        pos_c: Third position coordinates
    Returns:
        theta: Turning angle
    """
    vec_ab = Vec2d(pos_b.x - pos_a.x, pos_b.y - pos_a.y)
    norm_ab = vec_ab.get_length()

    vec_bc = Vec2d(pos_c.x - pos_b.x, pos_c.y - pos_b.y)
    norm_bc = vec_bc.get_length()

    dot_p = vec_ab.x * vec_bc.x + vec_ab.y * vec_bc.y

    # Nota: Evitar division por cero con np.finfo(float).eps
    cos_theta = dot_p / (norm_ab * norm_bc + np.finfo(float).eps)

    # Angle orientation
    cross_p = vec_ab.x * vec_bc.y - vec_ab.y * vec_bc.x
    orient = np.sign(cross_p)
    if orient == 0:
        orient = 1

    theta = np.arccos(np.around(cos_theta,4)) * orient
    return theta



## Actividad 1: Path Length - (BM1 vs BM2 vs CRW) (4 pts)

* Implementar función que genere **Brownian Motions** (BM) utilizando **pandas**.
* Implementar función que genere **Correlated Random Walks** (CRW) utilizando pandas.
* Implementar una función alternativa a las ya disponibles en los distintos modulos de python que calcule los valores de la curva de **path length** de una trayectoria.
* Guardar los valores de la métrica en un Data Frame de **pandas**.
* Visualizar con **plotly**.

In [10]:
# Load existing trajectories to test your implementation
# BM speed = 3
BM_2d_df_3 = pd.read_csv('/content/drive/MyDrive/trajectories/brownian_3.csv') #/trajectories/brownian_3.csv

# Load existing trajectories to test your implementation
# BM speed = 6
BM_2d_df_6 = pd.read_csv('/content/drive/MyDrive/trajectories/crw_6_6.csv')

# Load existing trajectories to test your implementation
CRW_2d_df_9 = pd.read_csv('/content/drive/MyDrive/trajectories/crw_6_9.csv')

In [11]:
# Define your function to compute path length for given trajectory
def path_length_curve(df):
  distances = [distance.euclidean(df.iloc[i-1][:2], df.iloc[i][:2]) for i in range(1,len(df))]
  return np.cumsum(distances)

pl_bm3 = path_length_curve(BM_2d_df_3)
pl_bm6 = path_length_curve(BM_2d_df_6)
pl_crw9 = path_length_curve(CRW_2d_df_9)

fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(pl_bm3)), y=pl_bm3, mode='lines', name='BM3 Path Length'))
fig.add_trace(go.Scatter(x=np.arange(len(pl_bm6)), y=pl_bm6, mode='lines', name='BM6 Path Length'))
fig.add_trace(go.Scatter(x=np.arange(len(pl_crw9)), y=pl_crw9, mode='lines', name='CRW9 Path Length'))
fig.update_layout(title='Path Length Curves', xaxis_title='Step', yaxis_title='Path Length')
fig.show()

In [12]:
def calculate_path_length(trajectory):
    distances = [distance.euclidean(trajectory.iloc[i-1,:2], trajectory.iloc[i,:2]) for i in range(1,len(trajectory))]
    path_length = np.cumsum(distances)
    path_length_df = pd.DataFrame({'Step': np.arange(1,len(path_length)+1), 'Path Length': path_length})
    return path_length_df

path_length_df_bm3 = calculate_path_length(BM_2d_df_3)

# Get Path length calling the function
PL_BM_3 = calculate_path_length(BM_2d_df_3)

# Get Path length calling the function
PL_BM_6 = calculate_path_length(BM_2d_df_6)

# Get Path length calling the function
PL_CRW_6 = calculate_path_length(CRW_2d_df_9)

In [13]:
# Plotting
# Init figure
fig_path_length = go.Figure()

# First trace BM1
fig_path_length.add_trace(go.Scatter(
    x=np.arange(len(PL_BM_3)) + 1,
    y=PL_BM_3,
    name='Path Length BM3',
    line=dict(color='blue', width=3),  # Estilo de línea azul
    showlegend=True
))

# Second trace BM2
fig_path_length.add_trace(go.Scatter(
    x=np.arange(len(PL_BM_6)) + 1,
    y=PL_BM_6,
    name='Path Length BM6',
    line=dict(color='orange', width=4),
    showlegend=True
))

# Third trace CRW
fig_path_length.add_trace(go.Scatter(
    x=np.arange(len(PL_CRW_6)) + 1,
    y=PL_CRW_6,
    name='Path Length CRW6',
    line=dict(color='purple', width=3),  # Estilo de línea morada
    showlegend=True
))
fig_path_length.update_layout(
    title='Comparación de Longitud de Trayectoria',
    xaxis_title='Paso',
    yaxis_title='Longitud de Trayectoria',
    legend_title='Tipo de Trayectoria',

    plot_bgcolor="white",  # Fondo blanco para mejorar contraste
    xaxis=dict(showline=True, showgrid=False, linecolor='black'),  # Estilo del eje X
    yaxis=dict(showline=True, showgrid=True, gridcolor='lightgrey', linecolor='black')  # Estilo del eje Y
)
fig_path_length.show()

## Actividad 2: Mean Squared Displacement - (Brownian vs CRW) (4 pts)

* Generar una trayectoria tipo **BM** y una **CRW**.
* Implementar una función que calcule los valores de la curva de **mean squared displacement** de una trayectoria.
* Guardar metricas en Pandas Data Frame.
* Visualizar con **plotly**.

In [14]:
# Load existing trajectories to test your implementation
# BM speed = 6
BM_2d_df_6 = pd.read_csv('/content/drive/MyDrive/trajectories/brownian_6.csv')

# Load existing trajectories to test your implementation
# CRW speed = 6, c = 0.9
CRW_2d_df_9 = pd.read_csv('/content/drive/MyDrive/trajectories/crw_6_9.csv')

In [15]:
# Show trajectories
# Init figure
fig_3d = go.Figure()

# Plot trajectory in 3-D space
fig_3d.add_trace(
    go.Scatter3d(x = BM_2d_df_6.x_pos,
                 y = BM_2d_df_6.y_pos,
                 z = BM_2d_df_6.index,
                 marker = dict(size=2),
                 line = dict(color='blue', width=2),
                 mode = 'lines',
                 name = 'BM 2d',
                 showlegend = True))


fig_3d.add_trace(
    go.Scatter3d(x = CRW_2d_df_9.x_pos,
                 y = CRW_2d_df_9.y_pos,
                 z = CRW_2d_df_9.index,
                 marker = dict(size=2),
                 line = dict(color='red', width=2),
                 mode = 'lines',
                 name = 'CRW 2d',
                 showlegend = True))

fig_3d.show()

In [16]:
def calculate_msd(trajectory):
    trajectory = trajectory.reset_index(drop=True)
    N = len(trajectory)

    msd_values = []
    for lag in range(1,N):
        squared_displacements = (trajectory.iloc[lag:]-trajectory.iloc[:-lag].values) ** 2
        msd = squared_displacements.sum(axis=1).mean()
        msd_values.append(msd)


    MSD_df = pd.DataFrame({'Lag': range(1,N), 'MSD':msd_values})
    return MSD_df

In [17]:
# Get Mean Squared Displacement calling the function
MSD_BM = calculate_msd(BM_2d_df_6[['x_pos', 'y_pos']])

# Get Mean Squared Displacement calling the function
MSD_CRW = calculate_msd(CRW_2d_df_9[['x_pos', 'y_pos']])

In [18]:
# Init figure
fig_path_length = go.Figure()

# first trace MSD_BM
fig_path_length.add_trace(go.Scatter(
    x=np.arange(len(PL_BM_3)) + 1,
    y=PL_BM_3,
    name='Path Length BM3',
    line=dict(color='green', width=3),  # Estilo de línea verde
    showlegend=True
))

fig_path_length.add_trace(go.Scatter(
    x=np.arange(len(PL_BM_6)) + 1,
    y=PL_BM_6,
    name='Path Length BM6',
    line=dict(color='red', width=4),
    showlegend=True
))
# Second trace MSD_CRW
fig_path_length.add_trace(go.Scatter(
    x=np.arange(len(PL_CRW_6)) + 1,
    y=PL_CRW_6,
    name='Path Length CRW6',
    line=dict(color='blue', width=3),  # Estilo de línea azul
    showlegend=True
))
fig_path_length.update_layout(
    title='Comparación de Longitud de Trayectoria',
    xaxis_title='Paso',
    yaxis_title='Longitud de Trayectoria',
    legend_title='Tipo de Trayectoria',

    plot_bgcolor="white",  # Fondo blanco para mejorar contraste
    xaxis=dict(showline=True, showgrid=False, linecolor='black'),  # Estilo del eje X
    yaxis=dict(showline=True, showgrid=True, gridcolor='lightgrey', linecolor='black')  # Estilo del eje Y
)

fig_path_length.show()

## Actividad 3: Turning-angle Distribution - (Dist. origen vs Dist. observada) (6 pts)

* Generar **CRWs** con dos exponentes diferentes
* Guardar trayectorias en **Pandas** Data Frame
* Implementar una función que calcule los valores de **turning angle** de una trayectoria.
* Comparar en gráfica distribución origen vs distribución observada (Histograma)
* Visualizar con **plotly**

In [8]:
# Load existing trajectories to test your implementation
# CRW speed = 6,
# wrapcauchy [c = 0.6]
CRW_2d_df_6 = pd.read_csv('/content/drive/MyDrive/trajectories/crw_6_6.csv') #/trajectories /content/drive/MyDrive/trajectories/crw_6_6.csv

# Load existing trajectories to test your implementation
# CRW speed = 6,
# wrapcauchy [c = 0.9]
CRW_2d_df_9 = pd.read_csv('/content/drive/MyDrive/trajectories/crw_6_9.csv')

In [19]:
# Define the function to compute Turning Angles for given trajectory
def turning_angle(pos_a, pos_b, pos_c):
    vec_ab = np.array(pos_b) - np.array(pos_a)
    norm_ab = np.linalg.norm(vec_ab)

    vec_bc = np.array(pos_c) - np.array(pos_b)
    norm_bc = np.linalg.norm(vec_bc)

    dot_p = np.dot(vec_ab, vec_bc)

    # Avoid division by zero with np.finfo(float).eps
    cos_theta = dot_p / (norm_ab * norm_bc + np.finfo(float).eps)

    # Angle orientation
    cross_p = np.cross(vec_ab, vec_bc)
    orient = np.sign(cross_p)
    if orient == 0:
        orient = 1

    theta = np.arccos(np.clip(cos_theta, -1, 1)) * orient
    return theta

In [21]:
# Define your function to compute Turning Angles for given trajectory
def turning_angle_trajectory(trajectory):
    angles = []
    for i in range(1, len(trajectory) - 1):
        angle = turning_angle(trajectory.iloc[i - 1], trajectory.iloc[i], trajectory.iloc[i + 1])
        angles.append(angle)
    return np.array(angles)

# Get Turning Angles calling the function for both datasets
ta_CRW_2d_df_6 = turning_angle_trajectory(CRW_2d_df_6)
ta_CRW_2d_df_9 = turning_angle_trajectory(CRW_2d_df_9)

from scipy.stats import cauchy
# Function to compute wrapped Cauchy PDF
def wrapped_cauchy_pdf(theta, rho):
    """
    Compute the PDF of the wrapped Cauchy distribution for a series of angles.
    """
    numerator = (1 - rho**2)
    denominator = (2 * np.pi * (1 + rho**2 - 2 * rho * np.cos(theta)))
    return numerator / denominator

# Compute the PDF for both parameters c = 0.6 and c = 0.9
theta_range = np.linspace(-np.pi, np.pi, 100)
pdf_c_6 = wrapped_cauchy_pdf(theta_range, 0.6)
pdf_c_9 = wrapped_cauchy_pdf(theta_range, 0.9)

In [22]:
# Histogram counts for scaling
count_6, bins_6 = np.histogram(ta_CRW_2d_df_6, bins=30, density=False)
count_9, bins_9 = np.histogram(ta_CRW_2d_df_9, bins=30, density=False)

# Determine the scaling factor for the PDF based on the maximum bin height of the histogram
scale_factor_peak_6 = max(count_6) / max(pdf_c_6)
scale_factor_peak_9 = max(count_9) / max(pdf_c_9)

# Apply this peak-based scaling factor
pdf_c_6_scaled_peak = pdf_c_6 * scale_factor_peak_6
pdf_c_9_scaled_peak = pdf_c_9 * scale_factor_peak_9

# Plotting with Plotly
fig = go.Figure()


In [23]:
# Check documentation
#https://plotly.com/python/histograms/

# PLot histogram
fig.add_trace(go.Histogram(x=ta_CRW_2d_df_6, name='Observado c=0.6', marker_color='green', opacity=0.6))
fig.add_trace(go.Histogram(x=ta_CRW_2d_df_9, name='Observado c=0.9', marker_color='purple', opacity=0.6))

# Add scaled PDF curves
fig.add_trace(go.Scatter(x=theta_range, y=pdf_c_6_scaled_peak, mode='lines', name='Cauchy c=0.6', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=theta_range, y=pdf_c_9_scaled_peak, mode='lines', name='Cauchy c=0.9', line=dict(color='red')))

# Update the layout
fig.update_layout(title='Histograma de Ángulos de Giro con PDF Envuelto de Cauchy', xaxis_title='Ángulo de Giro', yaxis_title='Cuenta',
                  bargap=0.2, barmode='overlay')

# Show the figure
fig.show()




## Actividad 4: Step-length Distribution - (Dist. origen vs Dist. observada) (6 pts)

* Implementar función que genere **Lévy Walks** (LW) utilizando pandas.
* Guardar trayectorias en Pandas Data Frame.
* Implementar una función que calcule los valores de **step lenght** de una trayectoria.
* Guardar trayectorias en **pandas** Data Frame.
* Obtener **Step-length** distribution.
* Comparar en gráfica distribución origen vs distribución observada (Histograma).
* Visualizar con plotly.

In [68]:
# Load existing trajectories to test your implementation
# Levy speed = 6
# levy_stable [alpha=1.0, beta=1.0, loc=3.0]
Levy_2d_df_1 = pd.read_csv('/trajectories/levy_6_1.csv')

# Load existing trajectories to test your implementation
# Levy speed = 6
# levy_stable [alpha=0.7, beta=1.0, loc=3.0]
Levy_2d_df_7 = pd.read_csv('/trajectories/levy_6_7.csv')

In [74]:
# Define your function to compute Step lengths for given trajectory

## start - Add your code here
def compute_step_lengths(trajectory):
    steps = []
    for i in range(len(trajectory) - 1):
        pos_a = Vec2d(trajectory.iloc[i]['x_pos'], trajectory.iloc[i]['y_pos'])
        pos_b = Vec2d(trajectory.iloc[i + 1]['x_pos'], trajectory.iloc[i + 1]['y_pos'])
        step_length = distance.euclidean((pos_a.x, pos_a.y), (pos_b.x, pos_b.y))
        steps.append(step_length)
    return pd.DataFrame(steps, columns=['Step Length'])

## end - Add your code here

In [75]:
# Get Step lengths calling the function
sl_Levy_2d_df_1 = compute_step_lengths(Levy_2d_df_1)

# Get Step lengths calling the function
sl_Levy_2d_df_7 = compute_step_lengths(Levy_2d_df_7)


In [76]:
# PLot histogram
fig_met_df_4 = go.Figure()

# Histogram step-length Levy_2d_df_1
## start - Add your code here
fig_met_df_4.add_trace(go.Histogram(x=sl_Levy_2d_df_1['Step Length'], name='Levy_2d_df_1'))
## end - Add your code here


# Histogram step-length Levy_2d_df_7
## start - Add your code here
fig_met_df_4.add_trace(go.Histogram(x=sl_Levy_2d_df_7['Step Length'], name='Levy_2d_df_7'))
## end - Add your code here


# Add origin distributions
## start - Add your code here

## end - Add your code here


origin_distribution = np.random.normal(loc=0, scale=1, size=1000)  # Mean = 0, Standard Deviation = 1

# Add origin distribution to the plot
fig_met_df_4.add_trace(go.Histogram(x=origin_distribution, name='Origin Distribution'))

# Add legend
fig_met_df_4.update_layout(
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

# Update layout
fig_met_df_4.update_layout(
    title="Step-length Distribution",
    xaxis_title="Step Length",
    yaxis_title="Frequency"
)

# Show plot
fig_met_df_4.show()